In [4]:
import numpy as np
var_idxs = np.load('data/multi_var_idxs.npy')
things_to_consider = np.sort(var_idxs[4000:])
len(things_to_consider)

224942

In [5]:
from datasets import SparseDataset

d = SparseDataset('all', 'multi')
x = d.inputs_npz[:, things_to_consider]
x.shape

(105942, 224942)

In [6]:
from sklearn.decomposition import TruncatedSVD

p = TruncatedSVD(n_components=4000)
p.fit(x)

TruncatedSVD(n_components=4000)

In [13]:
p2 = pk.load(open('pkls/multi_4000_pca.pkl', 'rb'))
p2.transform(x[2].toarray()).shape

(1, 4000)

In [7]:
import pickle as pk
pk.dump(p, open("multi_4000_pca.pkl","wb"))


In [11]:
for i in range(x.shape[0]):
    z = x[i].toarray()
    print(p.transform(z).shape)
    break

(1, 4000)


In [ ]:
import pickle
import pandas as pd

multi_keys = list(pd.read_hdf('data/train_multi_inputs.h5', start=0, stop=1).keys())
cite_keys = list(pd.read_hdf('data/train_multi_targets.h5', start=0, stop=1).keys())
with open('pkls/coding_map.pkl', 'rb') as f:
    rna_coders = pickle.load(f)
    
dna_coders = {k: [] for k in multi_keys}
for c in cite_keys:
    for d in rna_coders[c]:
        dna_coders[d].append(c)


In [ ]:
import numpy as np
from datasets import SparseDataset

idx = np.load('pkls/cite_best_idxs.npy')[4]
gene = cite_keys[idx]
dna_idxs = [multi_keys.index(s) for s in rna_coders[gene]]
dna_idx = dna_idxs[0]

# d = SparseDataset('train', 'multi')

x = d.inputs_npz[:, 170000:180000].toarray()
y = d.targets_npz[:, idx].toarray()

In [ ]:
from sklearn.linear_model import LinearRegression
import tqdm

xs = []
scores = []
for i in tqdm.tqdm(range(x.shape[1])):
    xs.append(170000 + i)
    x_i = x[:, i:i+1]
    p = LinearRegression()
    p.fit(x_i, y)
    scores.append(p.score(x_i, y))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(xs, scores)
plt.show()

In [ ]:
for v in dna_coders.values():
    print(v)

In [ ]:
import tqdm
already_seen = set()
ret = []  # list of tuples of sets
for k, v in tqdm.tqdm(rna_coders.items()):
    if k in already_seen or len(v) == 0: 
        continue
    
    curr = [set(), set()]
#     idx = 0
#     curr[idx].add(k)
#     queue = [(1-idx, v)]
#     while len(queue) > 0:
#         idx, thing = queue.pop()
#         for key in thing:
#             assert (idx == 0) == ('ENSG' in key)
#             curr[idx].add(key)
#             d = dna_coders if idx == 1 else rna_coders
#             l = d[key]
#             if len(l) > 0:
#                 queue.append((1 - idx, l))
        
    
    curr[0].add(k)
    for dna in v:
        curr[1].add(dna)
#         for rna in dna_coders[dna]:
#             curr[0].add(rna)
#             for dna_2 in rna_coders[rna]:
#                 curr[1].add(dna_2)
#                 for rna_2 in dna_coders[dna_2]:
#                     curr[0].add(rna_2)
#                     for dna_3 in rna_coders[rna_2]:
#                         curr[1].add(dna_3)
    for rna in curr[0]:
        already_seen.add(rna)
                
    ret.append(curr)

In [ ]:
import numpy as np
import tqdm
bad = True
while bad:
    print('before: {}'.format(len(ret)))
    bad = False
    idxs_to_look_through = list(range(len(ret)))
    for i, r1 in enumerate(tqdm.tqdm(ret)):
        if i not in idxs_to_look_through: continue
        for j in idxs_to_look_through:
            if i == j: continue
            r2 = ret[j]
            if not r1[0].isdisjoint(r2[0]) or not r1[1].isdisjoint(r2[1]): 
                bad = True 
                r1[0] = r1[0].union(r2[0])
                r1[1] = r1[1].union(r2[1])
                idxs_to_look_through.remove(j)
    ret = [ret[i] for i in idxs_to_look_through]
    print('after: {}'.format(len(ret)))

In [ ]:
rna_count = 0
dna_count = 0
for r in ret:
    rna_count += len(r[0])
    dna_count += len(r[1])
print(rna_count, dna_count)

In [ ]:
i = 0
for k in rna_coders.keys():
    if len(rna_coders[k]) == 1:
        i += 1
print(i)

i = 0
for k in dna_coders.keys():
    if len(dna_coders[k]) == 0:
        i += 1
print(i)

In [ ]:
t = 5
i = 0
for r in ret:
    if len(r[0]) >= t and len(r[1]) >= t:
        i += 1
print(i)

In [ ]:
import matplotlib.pyplot as plt
n_bins=30

lengths_rna = [len(r[0]) for r in ret]
lengths_dna = [len(r[1]) for r in ret]

fig, axs = plt.subplots(1, 2, sharey=True, tight_layout=True)

# We can set the number of bins with the *bins* keyword argument.
axs[0].hist(lengths_rna, bins=n_bins)
axs[1].hist(lengths_dna, bins=n_bins)
plt.show()

In [ ]:
lengths = [len(dna_coders[k]) for k in dna_coders.keys()]
lengths = np.argsort(lengths)[::-1]


In [ ]:
import pickle
with open('pkls/partition.pkl', 'wb') as f:
    pickle.dump(ret, f)

In [ ]:
champ = None
best = 0
for r in ret:
    if len(r[0]) > 4:
        if len(r[1]) > best:
            best = len(r[1])
            champ = r
rna_idxs = np.sort([cite_keys.index(k) for k in champ[0]])
dna_idxs = np.sort([multi_keys.index(k) for k in champ[1]])
len(rna_idxs)

In [ ]:
print(cite_keys)

In [ ]:
import numpy as np
import pickle
import pandas as pd
multi_keys = list(pd.read_hdf('data/train_multi_inputs.h5', start=0, stop=1).keys())
cite_keys = list(pd.read_hdf('data/train_multi_targets.h5', start=0, stop=1).keys())
with open('pkls/partition.pkl', 'rb') as f:
    ret = pickle.load(f)
    champ = None
    best = 0
    for r in ret:
        if len(r[0]) > 4:
            if len(r[1]) > best:
                best = len(r[1])
                champ = r
    rna_idxs = np.sort([cite_keys.index(k) for k in champ[0]])
    dna_idxs = np.sort([multi_keys.index(k) for k in champ[1]])
from datasets import H5Dataset, SparseDataset
from model import Model
import torch
import torch.nn as nn
import tqdm
from utils import focal_loss, device, negative_correlation_loss

# ------------------------------------- hyperparameters -------------------------------------------------

batch_size = 144

initial_lr = 0.02
lr_decay_period = 4
lr_decay_gamma = 0.5
weight_decay = 0.0004

num_epochs = 11
eval_every = 2
patience = 3
num_tries = 4

# --------------------------------------------------------------------------------------------------------

model = nn.Sequential(
    nn.Linear(186, 200),
    nn.ReLU(),
    nn.BatchNorm1d(200),
    nn.Dropout(0.1),
    nn.Linear(200, 50),
    nn.ReLU(),
    nn.Linear(50, 27)
)
model.to(device)
model.train()

# train_dataloader = SparseDataset('train', 'multi').get_dataloader(batch_size)
# val_dataloader = SparseDataset('val', 'multi').get_dataloader(batch_size)

optim = torch.optim.Adam(model.parameters(), initial_lr, weight_decay=weight_decay)
for _ in range(num_epochs):
    model.train()
    avg_loss = 0.0
    for x, day, y in tqdm.tqdm(train_dataloader):
        optim.zero_grad()
        out = model(x[:, dna_idxs].to(device))
        y = y[:, rna_idxs].to(device)
        loss = focal_loss(out, y)
        loss.backward()
        avg_loss += loss.cpu().item()
        optim.step()
    avg_loss /= len(train_dataloader)
    print('train', avg_loss)
    
    model.eval()
    avg_loss = 0.0
    for x, day, y in tqdm.tqdm(val_dataloader):
        out = model(x[:, dna_idxs].to(device))
        y = y[:, rna_idxs].to(device)
        loss = focal_loss(out, y)
        avg_loss += loss.cpu().item()
    avg_loss /= len(val_dataloader)
    print('val', avg_loss)


In [ ]:
from utils import negative_correlation_loss
model.eval()
for x, day, y in tqdm.tqdm(val_dataloader):
    out = torch.sigmoid(model(x[:, dna_idxs].to(device)))
    y = y[:, rna_idxs].to(device)
#     loss = torch.nn.functional.mse_loss(out, y)
#     print((out[0, :20] > 0.97).float())
    print(out[0, :20])
    print((y[0, :20] != 0).float())
    print(negative_correlation_loss(out, y))
    break